# Import required packages

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold,cross_validate, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
#from yellowbrick.classifier import ClassificationReport
import warnings
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")
from scipy import stats

# Data preprocessing stage - merging and cleaning of data

### Data Collection and Preprocessing

In [2]:
#merging all the required datasets - metabolomics dataset, drug response dataset and the metadata of the cell lines

data = pd.read_excel("D:\\GaTech\\Fall Sem\\BIOL 8901\\Metabolomic Project\\metabolomic_data.xlsx", sheet_name="1-clean data")
data.rename(columns = {'Unnamed: 0': 'ID'}, inplace = True)
last_column = data.iloc[:,-1].name
sample = pd.read_csv("D:\\GaTech\\Fall Sem\\BIOL 8901\\sample_info.csv")
sample.rename(columns = {'CCLE_Name':'ID'}, inplace=True)
merged_data = data.merge(sample, on='ID')
drug = pd.read_csv('D:\\GaTech\\Fall Sem\\BIOL 8901\\sanger-dose-response.csv')
drug.rename(columns={'ARXSPAN_ID':'DepMap_ID'}, inplace = True)
working_data = merged_data.merge(drug, on='DepMap_ID')

In [3]:
#since our data has a lot of NaN's in them, we can fill them using 0 (for now, just for a workaround)

working_data.fillna(0, inplace=True)

In [6]:
X1 = working_data.loc[(working_data['DRUG_NAME'] == 'CISPLATIN') & (working_data['lineage']=='ovary')]

In [7]:
'''
reset the index as once we take a subset of the main working_dataset, the indexes will get mixed up.
thus the indices need to be reset before we start working on the model
'''

X1.reset_index(inplace=True)

### Remove GDSC1 data (per GDSC instructions) and keep GDSC2 data only

In [9]:
#wherever there is any duplicate within a cell line's IC50 values, take the cell line that originates from the GDSC2 phase 
X_ = X1[~X1.duplicated(['DepMap_ID'], keep=False) | X1['DATASET'].eq('GDSC2')]

In [10]:
X_.reset_index(inplace=True)

In [11]:
#drop the metadata

X_ = X_.select_dtypes('float64')

In [12]:
X_.reset_index(inplace=True)

In [13]:
#scale the dataframe, at a quick glance the metabolite profiles are not scaled thus scaling of the dataframe is necessary

X_min = X_.min()
X_max = X_.max()
X_range = (X_max-X_min)
X_scaled = (X_-X_min)/(X_range)

In [14]:
#calculate the mean and standard deviation of the IC50_PUBLISHED

drug_mean = X_scaled['IC50_PUBLISHED'].mean()
drug_std = X_scaled['IC50_PUBLISHED'].std()
print(drug_mean, drug_std)

0.09638490494585468 0.21579685058728354


### Label Creation

In [15]:
#create labels

ic, labels = X_scaled['IC50_PUBLISHED'], []
for i in range(len(ic)):
    #if the IC50_PUBLISHED value is greater than the mean, add responsive label to the label list
    if ic[i] > drug_mean:
        labels.append('R')
    #if the IC50_PUBLISHED value is less than the mean, add non responsive label to the label list
    elif ic[i] < drug_mean:
        labels.append('NR')
        
y = pd.Series(labels)

In [16]:
y.value_counts()

NR    22
R      7
dtype: int64

In [17]:
X = X_.iloc[:, :225]
X.reset_index(inplace=True)

### Dimension reduction using Correlation

In [18]:
#calculate the correlation matrix of the metabolite dataframe
#choose the upper triangle of the correlation matrix
#create a list of features where the correlation value is >0.90
#this list contains the highly correlated features, which will be removed from the dataset

corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column]>0.90)]

In [19]:
len(to_drop)

30

In [20]:
#drop the list of highly correlated features computed above

X.drop(to_drop, axis=1, inplace=True)

In [21]:
X.shape, y.shape

((29, 196), (29,))

## Data Processing

### Feature Selection using Recursive Feature Elimination using Cross Validation

In [22]:
#run the RFECV model with estimator being Random Forest and StratifiedKFold cross validation with 5 folds.

rfecv = RFECV(estimator = RandomForestClassifier(random_state=101), step=1, cv=StratifiedKFold(5), scoring='accuracy')

In [23]:
#fit the X,y to the RFECV model

rfecv.fit(X, y)

RFECV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
      estimator=RandomForestClassifier(random_state=101), scoring='accuracy')

In [24]:
#choose the features that are selected by the RFECV model

selected_features = rfecv.get_support(1)

In [25]:
#select a subset dataframe that contains only the "optimal" metabolic features returned from the RFECV model

X3 = X[X.columns[selected_features]]
X3.shape

(29, 163)

### Random Forest 

### Model training and Classification

In [26]:
#using classification_report metrics, run a prediction model using StratifiedKFold cross_validation with k=5 folds
#model being used as the classifier is Random Forest

kf3 = StratifiedKFold(n_splits = 5, shuffle=False)
model=RandomForestClassifier()
i=1
dfs = []
for train_index, test_index in kf3.split(X3, y):
    #select train and test datasets from X and y
    X_train, X_test = X3.iloc[train_index], X3.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #train the model
    model.fit(X_train, y_train)
    #predict the test dataset
    predicted = model.predict(X_test)
    #print the classification score report
    report = classification_report(y_test, predicted, output_dict = True)
    df = pd.DataFrame(report).transpose()
    dfs.append(df)
    i+=1
results_df = pd.concat(dfs)

In [27]:
results_df

,precision,recall,f1-score,support
NR,0.833333,1.000000,0.909091,5.000000
R,0.000000,0.000000,0.000000,1.000000
accuracy,0.833333,0.833333,0.833333,0.833333
macro avg,0.416667,0.500000,0.454545,6.000000
weighted avg,0.694444,0.833333,0.757576,6.000000
NR,0.833333,1.000000,0.909091,5.000000
R,0.000000,0.000000,0.000000,1.000000
accuracy,0.833333,0.833333,0.833333,0.833333
macro avg,0.416667,0.500000,0.454545,6.000000
weighted avg,0.694444,0.833333,0.757576,6.000000


In [25]:
#results_df.to_csv("CISPLATIN_Results_RF.tsv", sep="\t")

### SVM

### Model training and classification

In [28]:
#using classification_report metrics, run a prediction model using StratifiedKFold cross_validation with k=5 folds
#model being used as the classifier is Support Vector Machine

kf5 = StratifiedKFold(n_splits = 5, shuffle=False)
model2=svm.SVC(kernel='linear', C=100)
i=1
dfs_svm = []
for train_index, test_index in kf5.split(X3, y):
    #select train and test datasets from X and y
    X_train, X_test = X3.iloc[train_index], X3.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #train the model
    model2.fit(X_train, y_train)
    #predict the test dataset
    predicted2 = model2.predict(X_test)
    #print the classification score report
    report = classification_report(y_test, predicted2, output_dict = True)
    df = pd.DataFrame(report).transpose()
    dfs_svm.append(df)
    i+=1
results_df_svm = pd.concat(dfs_svm)

In [29]:
results_df_svm

,precision,recall,f1-score,support
NR,1.000000,0.800000,0.888889,5.000000
R,0.500000,1.000000,0.666667,1.000000
accuracy,0.833333,0.833333,0.833333,0.833333
macro avg,0.750000,0.900000,0.777778,6.000000
weighted avg,0.916667,0.833333,0.851852,6.000000
NR,0.833333,1.000000,0.909091,5.000000
R,0.000000,0.000000,0.000000,1.000000
accuracy,0.833333,0.833333,0.833333,0.833333
macro avg,0.416667,0.500000,0.454545,6.000000
weighted avg,0.694444,0.833333,0.757576,6.000000


### AdaBoost

### Model training and classification

In [30]:
#using classification_report metrics, run a prediction model using StratifiedKFold cross_validation with k=5 folds
#model being used as the classifier is AdaBoost Algorithm

kf5 = StratifiedKFold(n_splits = 5, shuffle=False)
model3=AdaBoostClassifier(n_estimators=500, learning_rate=0.8)
i=1
dfs_adb = []
for train_index, test_index in kf5.split(X3, y):
    #select train and test datasets from X and y
    X_train, X_test = X3.iloc[train_index], X3.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #train the model
    model3.fit(X_train, y_train)
    #predict the test dataset
    predicted3 = model3.predict(X_test)
    #print the classification score report
    report = classification_report(y_test, predicted3, output_dict = True)
    df = pd.DataFrame(report).transpose()
    dfs_adb.append(df)
    i+=1
results_df_adb = pd.concat(dfs_adb)

In [31]:
results_df_adb

,precision,recall,f1-score,support
NR,0.800000,0.800000,0.800000,5.000000
R,0.000000,0.000000,0.000000,1.000000
accuracy,0.666667,0.666667,0.666667,0.666667
macro avg,0.400000,0.400000,0.400000,6.000000
weighted avg,0.666667,0.666667,0.666667,6.000000
NR,0.800000,0.800000,0.800000,5.000000
R,0.000000,0.000000,0.000000,1.000000
accuracy,0.666667,0.666667,0.666667,0.666667
macro avg,0.400000,0.400000,0.400000,6.000000
weighted avg,0.666667,0.666667,0.666667,6.000000


## Voting Classifier

### Model training and Classification

In [32]:
estimator = []
estimator.append(('DTC', DecisionTreeClassifier()))
estimator.append(('SVC', svm.SVC(kernel='linear', gamma = 'auto', probability = True)))
estimator.append(('KNN', KNeighborsClassifier(n_neighbors=3)))
kf5 = StratifiedKFold(n_splits = 5, shuffle=False)
model3=AdaBoostClassifier(n_estimators=300, learning_rate=1.0)
i=1
dfs_vc = []
for train_index, test_index in kf5.split(X3,y):
    #select train and test datasets from X and y
    X_train, X_test = X3.iloc[train_index], X3.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #train the model
    vc = VotingClassifier(estimators = estimator, voting='hard').fit(X_train, y_train)
    #predict the test dataset
    predicted3 = vc.predict(X_test)
    #print the classification score report
    report = classification_report(y_test, predicted3, output_dict = True)
    df = pd.DataFrame(report).transpose()
    dfs_vc.append(df)
    i+=1
results_df_vc = pd.concat(dfs_vc)

In [33]:
results_df_vc

,precision,recall,f1-score,support
NR,0.800000,0.800000,0.800000,5.000000
R,0.000000,0.000000,0.000000,1.000000
accuracy,0.666667,0.666667,0.666667,0.666667
macro avg,0.400000,0.400000,0.400000,6.000000
weighted avg,0.666667,0.666667,0.666667,6.000000
NR,0.833333,1.000000,0.909091,5.000000
R,0.000000,0.000000,0.000000,1.000000
accuracy,0.833333,0.833333,0.833333,0.833333
macro avg,0.416667,0.500000,0.454545,6.000000
weighted avg,0.694444,0.833333,0.757576,6.000000
